# Data Structures and Algorithms
## Group 4
## Final Project
## Stock Market Clustering


Load packages and data as pandas data frame. Define graphs and directed graphs.

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import csv

##### Read company names into a dictionary
#def readNamesIntoDict():
#    d = dict()
#    input_file = csv.DictReader(open("SP_500_firms.csv"))
#    for row in input_file:
#        #print(row)
#        d[row['Symbol']] = [row['Name'],row['Sector']]
#   return d
#
#namesDict = readNamesIntoDict()
#
#compNames = namesDict.keys()
#
#
###### Prices into standarad Python data structures
#
## Read prices into dictionary of lists
##
#def readPricesIntoDict():
#    input_file = csv.DictReader(open('SP_500_close_2015.csv', 'r')) 
#    d = dict()
#    for row in input_file:
#        for column, value in row.items():
#            d.setdefault(column, []).append(value)
#    return d


class Digraph():
    #edges is a dict mapping each node to a dictionary of edges starting from it
    def __init__(self,filename = None):
        self.edges = {}
        self.numEdges = 0
    
    def addNode(self,node):
        # add a vertex to graph (based on key value)       
        self.edges[node] = set()

    def addEdge(self,src,dest,weight):
        # add the (v,w) edge, making sure the two nodes exist
        if not self.hasNode(src): 
            self.addNode(src)
            self.edges[src] = {}
        if not self.hasNode(dest): 
            self.addNode(dest)
            self.edges[dest] = {}
        if not self.hasEdge(src, dest):
            self.numEdges += 1
            self.edges[src][dest] = weight
  
    def childrenOf(self, v):
        # Returns a node's children
        return self.edges[v].items()
        
    def hasNode(self, v):
        return v in self.edges
        
    def hasEdge(self, v, w):
        return w in self.edges[v]

    def listEdges(self):
        ll = []
        for src,values in self.edges.items():
            for dest,weight in values.items():
                ll.append([src,dest,weight])
        return ll
    
    def __str__(self):
        result = ''
        for src in self.edges:
            for dest,weight in self.edges[src].items():
                result = result + src + '->'\
                         + dest + ', length ' + str(weight) + '\n'
        return result[:-1] # omit final newline

class Graph(Digraph):
    """ Undirected graph: two one-way edges for every edge
        """
    def addEdge(self, src, dest,weight):
        Digraph.addEdge(self, src, dest,weight)
        Digraph.addEdge(self, dest, src,weight)
        
        
        
filename = 'SP_500_close_2015.csv'
priceData = pd.read_csv(filename,index_col = 0)


for company in priceData:
	companyData = priceData[company]
	pct_change = companyData[1:]/companyData[:-1]
	#print(pct_change)

percent_change = priceData.pct_change()


## 1. Calculating Returns

In [4]:
def returns (priceList):
    ret = [0]
    for i in range(1,len(priceList)):
        day = (float(priceList[i])-float(priceList[i-1]))/float(priceList[i-1])
        ret.append(day)
    return ret

def setReturns (prices):
    ret = {}
    for key, value in prices.items():
        if key == 'Date':
            pass
        else: 
            ret[key]=returns(value)
    return ret

prices = readPricesIntoDict()
Returns = setReturns(prices)

Evaluations go here

## 2. Correlations

In [20]:
def corr2(x,y):
    xy = sum([a*b for a,b in zip(x,y)])
    x2 = sum([i**2 for i in x])
    y2 = sum([i**2 for i in y])
    n = len(x)
    numer = (n*xy - sum(x)*sum(y))
    denom = ((n*x2 - sum(x)**2)**(1/2) * (n*y2 - sum(y)**2)**(1/2))
    correlation = numer/denom
    return correlation

correlations = {}

for i in percent_change:
    correlations[i] = {}

for i in correlations:    
    for j in percent_change:
        correlations[i][j]=[]
    
for company1 in percent_change:
    for company2 in percent_change:
        if not correlations[company1][company2]:
            x=percent_change[company1]        
            y=percent_change[company2]
            if company1==company2:
                correlations[company1][company2]=1
                correlations[company2][company1]=1
            else:
                value = corr2(x,y)
                correlations[company1][company2]=value
                correlations[company2][company1]=value



Evaluations go here

## 3. Clustering Algorithm

In [23]:
def mergeSort(array):
	if len(array) > 1:
		mid = len(array) //2
		left = array[:mid]
		right = array [mid:]
		mergeSort(left)
		mergeSort(right)

		i = 0
		j = 0
		k = 0
		while i < len(left) and j < len(right):
			if left[i][2] > right[j][2]:
				array[k] = left[i]
				i = i + 1
			else:
				array[k] = right[j]
				j = j + 1
			k = k+1
		while i < len(left):
			array[k] = left[i]
			i += 1
			k += 1
		while j < len(right):
			array[k] = right[j]
			j += 1
			k += 1
	return(array)
sortedWeights = mergeSort(input)
print("sortedweights",sortedWeights)

graph = Digraph()
for x in sortedWeights:
	graph.addEdge(x[0],x[1],x[2])
#print(graph)


nodePointers = {src:src for src in graph.edges}
nodeBottom = {src:True for src in graph.edges}
nodeTop = {src:True for src in graph.edges}

def findbottom(node):
    source = node
    destination = nodePointers[source]
    while destination != source:
        source = destination
        destination = nodePointers[source]
    return destination



def pointers (j):
    counter = 0
    for k in sortedWeights:
        if counter < j:
            if nodePointers[k[0]] == k[0]:
                print (nodeBottom[k[1]], k[1])
                if nodeBottom[k[1]]:
                    nodePointers[k[0]] = k[1]
                    nodeBottom[k[0]] = False
                    nodeTop[k[1]] = False
                else:
                    bottom = findbottom(k[1])
                    nodePointers[bottom] = k[0]
                    nodeBottom[bottom] = False
    
    
    
                print(nodePointers)
                print(nodeBottom)
                print(nodeTop)
                
            counter += 1


sortedweights [('c', 'd', 0.95), ('a', 'b', 0.8), ('b', 'c', 0.68), ('a', 'c', 0.63), ('b', 'd', 0.32), ('a', 'd', 0.05)]
True d
{'d': 'd', 'c': 'd', 'a': 'a', 'b': 'b'}
{'d': True, 'c': False, 'a': True, 'b': True}
{'d': False, 'c': True, 'a': True, 'b': True}
True b
{'d': 'd', 'c': 'd', 'a': 'b', 'b': 'b'}
{'d': True, 'c': False, 'a': False, 'b': True}
{'d': False, 'c': True, 'a': True, 'b': False}
False c
{'d': 'b', 'c': 'd', 'a': 'b', 'b': 'b'}
{'d': False, 'c': False, 'a': False, 'b': True}
{'d': False, 'c': True, 'a': True, 'b': False}


## 4.2 Alternative Clustering Methods

### Using correlations
Suppose we are solely interested in clustering the stocks based on correlation. Then we might rely on other methods.

#### Agglomerative heirachical clustering
An example that is very similar to the one we have done is agglomerative clustering. The main difference is that the algorithm does not terminate after $k$ steps. Instead, it keeps going until all clusters are merged. At each step, we record the "distance" between clusters when merging. When the algorithm terminates, we need only examine the "distances" between the clusters and choose a maximimum "distance", then break the links between all clusters that were at least that far apart before merging.

The advantage of this method is that we do not need to determine k before we begin the algorithm, we need only run the algorithm once and decide what $k$ should be after. Alternatively, plotting a dendogram would give us a visual aid to determine our cut off distance. Furthermore, by changing the way we determine distance between clusters, we could achieve much tighter clusters.

The disadvantage is that there is a need to decide how to calculate "distance" between clusters. The distance between any two nodes is straightforward, the correlation between the two nodes. Now consider that we have two clusters of size $n$ and $m$. In this case how do we define distance? Do we use single linkage, the shortest distance between any two nodes from both clusters? Or complete linkage, the farthest distance?

In our case, we want to cluster stocks such that they are all closely related. It would make sense to use average linkage, the average correlation between all pairwise stocks. This helps to enforce clusters that are all close to each other, with no node in a cluster being too far away from the other. 

Another major disadvantage to using heirachical clustering is the need to update our distances at every iteration. Adding on the fact that calculating the distance between two clusters is O(nm), the time complexity of carrying out this operation would be much greater. 


#### $k$ medoids clustering

The basic idea of $k$ medoids clustering in this context is to pick a pre-determined $k$ nodes as "centers" , put them into $k$ different clusters, then assign every other node to the nearest center. In a $k$-means algorithm, in the next iteration, we would determine a "center". This is done by finding the means of each cluster. However, this would require us to provide co-ordinates, which is something we do not have for this problem. Instead, the $k$ medoids algorithm simply picks the node that is closest to the center to be the new center. For every node, we calculate the sum of its distance ($1$-correlation) to every other member of its cluster. Then for each cluster, pick the node with the smallest sum of distance ot be the new medoid, then carry out the next iteration.

1. Randomly assign $k$ nodes as medoids of $k$ clusters
2. Assign every other node to cluster of its closest medoid
3. For each cluster:
  1. For each node in cluster:
    1. Calculate sum of distance between node and all other nodes in cluster
  2. Set node with smallest sum of distance as new medoid
4. Repeat steps 2-3

Calculating the sum of distance for every node in the graph is an expensive operation that runs in O($n^2$) time, where $n$ is the maximum number of nodes in a cluster. Instead of calculating all sum of distance, we can instead use a greedy approach, step 3-1 then becomes:

1. For each node in cluster:
  1. Calculate sum of distance
  2. If new sum of distance < old sum of distance:
    1. Set node as new medoid
    2. break
      
The main advantage of this method over the others is its ability to re-allocate nodes into different clusters. For the given algorithm and heirachical clustering, once a node is in a cluster, it will never be re-allocated. With $k$ medoids however, the algorithm will constantly re-assign nodes into the most appropriate cluster.

The disadvantage is the need to pre-specify the number of clusters. Before running any algorithms, it is hard to determine how many clusters there should be.

To counter this problem, we can consider a hybrid of both solutions. First, we do a heirachical cluster to determine the number of clusters, $k$. Using this result, we then run the $k$-medoids algorithm.

### Using all data

#### Artificial Neural Network
Something something
